In [3]:
import os, sys, inspect
# conabio_ml_vision
from conabio_ml_vision.datasets import ImageDataset
from conabio_ml_vision.trainer.model import run_megadetector_inference
from conabio_ml_vision.datasets import ImagePredictionDataset
from conabio_ml_vision.utils.scripts import create_obj_level_ds_from_dets_and_anns

In [5]:
# paths
missouri_ds_folder = os.path.join('/LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/data')
results_path = os.path.join('/LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles')
images_ds_csv = os.path.join(results_path, 'imgs_ds.csv')
out_detections_csv = os.path.join(results_path,'detections.csv')
crops_ds_csv = os.path.join(results_path, 'crops_ds.csv')
crops_folder = os.path.join(results_path, 'crop_images')
megadet_model_path = '/LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/files/megadetector_v5a.pt'
frames_w_bboxes_folder = os.path.join(results_path, 'frames_w_bboxes_folder')


In [6]:
# constantes
MIN_SCORE_DETS = 0.2

In [4]:
imgs_missouri_ds = ImageDataset.from_folder(missouri_ds_folder,
                                               label_by_folder_name=True,
                                               split_by_folde=True)




imgs_missouri_ds.to_csv(images_ds_csv,remove_label_to_item_path=True)
img_ds = ImageDataset.from_csv(images_ds_csv, images_dir=missouri_ds_folder,append_label_to_item_path=True, 
                               clean_cat_names=False)

2022-10-05 20:43:07,390 [conabio_ml_vision.datasets.media] [DEBUG]  No valid folder names for partitions. Set split_by_folder to False
2022-10-05 20:43:07,401 [conabio_ml.datasets.dataset] [INFO ]  Creating dataset from folder: /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/data
2022-10-05 20:43:07,754 [conabio_ml.datasets.dataset] [DEBUG]  487 files found for extensions .JPG, .jpg, .bmp, .png in /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/data
2022-10-05 20:43:07,839 [conabio_ml.datasets.dataset] [WARNI]  Some categories of the labelmapare not contained in the dataset
2022-10-05 20:43:07,840 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 13 categories (0: Collared_Peccary, 1: Common_Opossum, 2: European_Hare, 3: Ocelot, 4: Paca, 5: Red_Brocket_Deer, 6: Red_Fox, 7: Red_Squirrel, 8: Roe_deer, 9: Spiny_Rat...)
2022-10-05 20:43:07,891 [conabio_ml_vision.datasets.media] [INFO ]  487  validated items found
2022

In [5]:
run_megadetector_inference(dataset=imgs_missouri_ds,
                            out_predictions_csv=out_detections_csv,
                                   images_dir=missouri_ds_folder,
                                   model_path=megadet_model_path,
                                   min_score_threshold=MIN_SCORE_DETS,
                                   num_tasks=None,
                                   task_num=None)

2022-10-05 20:43:14,550 [conabio_ml_vision.trainer.model] [INFO ]  Using labelmap for prediction: [{1: 'Animal'}, {2: 'Person'}, {3: 'Vehicle'}]...
Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
  0%|          | 0/487 [00:00<?, ?it/s]

Loaded 487 image filenames from list file /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/__megadetector_imgs.json
PyTorch reports 0 available CUDA devices
GPU available: False
Using PyTorch version 1.10.1
Loaded model in 3.44 seconds
Loaded model in 3.44 seconds


100%|██████████| 487/487 [09:59<00:00,  1.23s/it]


Finished inference for 487 images in 10 minutes and 7.01 seconds
Output file saved at /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/__megadetector_out.json
Done!


2022-10-05 20:53:22,988 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 1 categories (1: animal)
2022-10-05 20:53:23,015 [conabio_ml_vision.datasets.media] [INFO ]  602  validated items found
2022-10-05 20:53:23,042 [conabio_ml_vision.utils.dataset_utils] [INFO ]  Getting images sizes from stored files...
2022-10-05 20:53:23,677 [conabio_ml.datasets.dataset] [INFO ]  CSV file /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/detections.csv created


In [15]:
def crop_images(detections_csv,frames_folder,crops_ds_csv,crops_folder):
    """
    Crop the images using bounding boxes coordinates generated by Megadetector's inference.  
    Save the resulted ImageDataset as a csv file 
    
    Parameters
    ----------
    detections_csv: string
        path to csv file with detections from megadetector. 
    frames_folder: string
        path to folder with video frames
    crops_ds_csv: string
        path to csv file with crop images dataset
    crops_folder: string
        path to folder with crop images
    
    
    Returns
    -------
    crops_ds: ImageDataset
        ImageDataset object of crop images
    """
    if not os.path.isfile(crops_ds_csv):
        dets_ds = ImagePredictionDataset.from_csv(detections_csv, images_dir=frames_folder)
        crops_ds = dets_ds.create_classif_ds_from_det_crops(
                    dest_path=crops_folder,
                    score_threshold=0.1)
        crops_ds.to_csv(crops_ds_csv)
    else:
        crops_ds = ImageDataset.from_csv(crops_ds_csv,images_dir=crops_folder)
    return crops_ds

In [6]:
# para que todos apunten al mismo lugar:

dets_ds = ImagePredictionDataset.from_csv(source_path=out_detections_csv,images_dir=missouri_ds_folder)

2022-10-05 21:03:17,604 [conabio_ml.datasets.dataset] [INFO ]  Creating dataset from CSV file: /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/detections.csv
2022-10-05 21:03:17,617 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 1 categories (1: animal)
2022-10-05 21:03:17,662 [conabio_ml_vision.datasets.media] [INFO ]  602  validated items found


In [83]:
#dets_ds.get_rows().item[0]
#img_ds.get_rows().item[0]

'/LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/data/Roe_deer/SEQ76392_IMG_0014.JPG'

In [7]:
animal_dets_obj_level_ds = create_obj_level_ds_from_dets_and_anns(detections=dets_ds,
                                           annotations=img_ds,
                                           images_dir=missouri_ds_folder,
                                           inherit_fields=[],
                                           inherit_partitions=False,
                                           annot_categories=None,
                                           exclude_annot_categories=None,
                                           det_categories=None,
                                           exclude_det_categories=None,
                                           min_score_detections=0.001,
                                           include_detection_score=True)

2022-10-05 21:03:22,473 [conabio_ml.datasets.dataset] [WARNI]  Some categories of the labelmapare not contained in the dataset
2022-10-05 21:03:22,475 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 13 categories (0: Collared_Peccary, 1: Common_Opossum, 2: European_Hare, 3: Ocelot, 4: Paca, 5: Red_Brocket_Deer, 6: Red_Fox, 7: Red_Squirrel, 8: Roe_deer, 9: Spiny_Rat...)
2022-10-05 21:03:22,494 [conabio_ml_vision.datasets.media] [INFO ]  487 not validated items found
2022-10-05 21:03:22,522 [conabio_ml.datasets.dataset] [WARNI]  Some categories of the labelmapare not contained in the dataset
2022-10-05 21:03:22,523 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 13 categories (1: Collared_Peccary, 2: Common_Opossum, 3: European_Hare, 4: Ocelot, 5: Paca, 6: Red_Brocket_Deer, 7: Red_Fox, 8: Red_Squirrel, 9: Roe_deer, 10: Spiny_Rat...)


In [8]:
animal_dets_obj_level_ds.get_rows()

,item,label,bbox,id,image_id,score_det
0,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Roe_deer,"199,689,554,359",b681f75a-5ae6-4506-a2fc-3ef8f5cbfc76,SEQ76392_IMG_0014,0.930
1,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Roe_deer,"177,748,145,112",53eb7608-f294-464e-96d6-692d1c3e3ee5,SEQ76392_IMG_0011,0.859
2,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Roe_deer,"296,697,491,354",3ed386bc-2d09-48dc-9371-b4cbf7329181,SEQ76392_IMG_0011,0.917
3,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Roe_deer,"555,756,88,140",0e36237c-6ca8-460b-8810-33fa961dbace,SEQ76392_IMG_0018,0.766
4,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Roe_deer,"0,716,328,363",80af26be-c97a-4df8-9d8f-351c5a990333,SEQ76392_IMG_0018,0.950
...,...,...,...,...,...,...
597,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Spiny_Rat,"1063,989,228,454",a6306850-3af2-44d5-acfb-dc9aea627dd9,SEQ80957_IMG_0005,0.951
598,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Spiny_Rat,"1000,773,176,479",b2943583-3975-4ace-b4b7-5ae363582b35,SEQ80957_IMG_0007,0.941
599,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Spiny_Rat,"990,969,183,233",5bedbb38-8f57-45dc-8096-eded4387149b,SEQ80957_IMG_0008,0.935
600,/LUSTRE/users/ecoinf_admin/conabio_ml_vision/e...,Spiny_Rat,"1345,1192,616,311",a07c9d86-7c17-4311-9d51-8ae84170bbb3,SEQ80957_IMG_0003,0.938


Ese función te regresa una instancia de ImageDataset con la que después llamas a create_classif_ds_from_bboxes_crops  para que haga los recortes, y luego haces un to_folder para que guarde los crops en las carpetas por cada label

In [9]:
crops_ds = animal_dets_obj_level_ds.create_classif_ds_from_bboxes_crops(
    dest_path=None, allow_label_empty=True) # esto ayuda cuando tenemos los resultados del megadetector y mis etiquetas del dataset
crops_ds.to_csv(crops_ds_csv)
crops_ds.to_folder(crops_folder)

2022-10-05 21:03:29,138 [conabio_ml_vision.datasets.datasets] [INFO ]  Creating classification dataset from detection bboxes.
2022-10-05 21:03:29,176 [conabio_ml_vision.datasets.datasets] [INFO ]  Generating 602 crops from detection dataset
2022-10-05 21:03:31,439 [conabio_ml.datasets.dataset] [WARNI]  Some categories of the labelmapare not contained in the dataset
2022-10-05 21:03:31,440 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 13 categories (0: Collared_Peccary, 1: Common_Opossum, 2: European_Hare, 3: Ocelot, 4: Paca, 5: Red_Brocket_Deer, 6: Red_Fox, 7: Red_Squirrel, 8: Roe_deer, 9: Spiny_Rat...)
2022-10-05 21:03:31,485 [conabio_ml_vision.datasets.media] [INFO ]  602  validated items found
2022-10-05 21:03:31,526 [conabio_ml.datasets.dataset] [INFO ]  CSV file /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/crops_ds.csv created
2022-10-05 21:03:31,527 [conabio_ml.datasets.dataset] [INFO ]  Writing the elements of the d

'/LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/crop_images'

Sí, aunque habría que asegurarnos que no haya recortes de una misma imagen en particiones distintas
Eso se puede hacer agrupando por el campo image_id a la hora de hacer el split, eso se le pasa en el parámetro group_by_field

In [1]:
from conabio_ml_vision.utils.scripts import visualize_detections_in_ds

In [9]:
imgs_ds = ImageDataset.from_csv(images_ds_csv, images_dir=missouri_ds_folder,append_label_to_item_path=True, 
                               clean_cat_names=False)

2022-10-06 23:22:36,055 [conabio_ml.datasets.dataset] [INFO ]  Creating dataset from CSV file: /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/imgs_ds.csv
2022-10-06 23:22:36,067 [conabio_ml.datasets.dataset] [WARNI]  Some categories of the labelmapare not contained in the dataset
2022-10-06 23:22:36,068 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 13 categories (0: Collared_Peccary, 1: Common_Opossum, 2: European_Hare, 3: Ocelot, 4: Paca, 5: Red_Brocket_Deer, 6: Red_Fox, 7: Red_Squirrel, 8: Roe_deer, 9: Spiny_Rat...)
2022-10-06 23:22:36,110 [conabio_ml_vision.datasets.media] [INFO ]  487  validated items found


In [10]:
visualize_detections_in_ds(images_ds=imgs_ds, 
                                   detections=out_detections_csv,
                                   images_dir=missouri_ds_folder,
                                   dest_dir=frames_w_bboxes_folder,
                                   min_score_dets=MIN_SCORE_DETS,
                                   score_thres_colors=0.95)

2022-10-06 23:22:40,187 [conabio_ml_vision.utils.scripts] [INFO ]  Visualizing detections of images in folder /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/frames_w_bboxes_folder
2022-10-06 23:22:40,191 [conabio_ml.datasets.dataset] [INFO ]  Creating dataset from CSV file: /LUSTRE/users/ecoinf_admin/conabio_ml_vision/examples/classification/missouri/missourifiles/detections.csv
2022-10-06 23:22:40,201 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 1 categories (1: animal)
2022-10-06 23:22:40,244 [conabio_ml_vision.datasets.media] [INFO ]  602  validated items found
2022-10-06 23:22:40,248 [conabio_ml.datasets.dataset] [DEBUG]  Assigning labelmap with 1 categories (1: animal)
